In [6]:
from PIL import Image
from multiprocessing import Pool

import os

LOAD_RAW_FOLDER = "crawlers/images/full/"
SAVE_4K_FOLDER = "./data/4k/"
SAVE_1080_FOLDER = "./data/1080/"
SAVE_540_FOLDER = "./data/540/"
SAVE_270_FOLDER = "./data/270/"
os.makedirs(SAVE_4K_FOLDER, exist_ok=True)
os.makedirs(SAVE_1080_FOLDER, exist_ok=True)
os.makedirs(SAVE_540_FOLDER, exist_ok=True)
os.makedirs(SAVE_270_FOLDER, exist_ok=True)

skip_existing = True

In [2]:
for f in os.listdir(LOAD_RAW_FOLDER):
    save_path = os.path.join(SAVE_4K_FOLDER, f)
    if skip_existing and os.path.exists(save_path):
        continue

    im = Image.open(os.path.join(LOAD_RAW_FOLDER, f))
    width, height = im.size

    target_width = 3840
    target_height = 2160
    im1 = None
    im2 = None
    if width > target_width:
        left = (width - target_width) / 2
        im1 = im.crop((left, 0, width - left, height))
    elif target_width > width:
        left = (target_width - width) / 2
        im1 = Image.new(im.mode, (target_width, height), (0, 0, 0))
        im1.paste(im, (left, 0))
    else:
        im1 = im

    if height > target_height:
        top = (height - target_height) / 2
        im2 = im1.crop((0, top, target_width, height - top))
    elif target_height > height:
        top = (target_height - height) / 2
        im2 = Image.new(im1.mode, (target_width, target_height), (0, 0, 0))
        im2.paste(im1, (0, top))
    else:
        im2 = im1

    im2.save(save_path)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/PIL/Image.py:2855: DecompressionBombWarning: Image size (150969345 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/PIL/Image.py:2855: DecompressionBombWarning: Image size (120744088 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/PIL/Image.py:2855: DecompressionBombWarning: Image size (126570000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/PIL/Image.py:2855: DecompressionBombWarning: Image size (144000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/PIL/Image.py:2855: DecompressionBombWarning: Image size 

In [3]:
for f in os.listdir(SAVE_4K_FOLDER):
    if f.startswith('.'):
        continue
    save_path = os.path.join(SAVE_540_FOLDER, f)
    if skip_existing and os.path.exists(save_path):
        continue

    im = Image.open(os.path.join(SAVE_4K_FOLDER, f))
    target_size = (960, 540)
    im = im.resize(target_size)
    im.save(save_path, "PNG")

In [12]:

# clip = [(0, 0, 1920, 1080), (1920, 0, 3840, 1080), (0, 1080, 1920, 2160), (1920, 1080, 3840, 2160)]

def f_1080(f):
    if f.startswith('.'):
        return
    
    """im = Image.open(os.path.join(SAVE_4K_FOLDER, f))
    for i in range(0, 4):
        save_path = os.path.join(SAVE_1080_FOLDER, str(i) + "_" + f)
        if skip_existing and os.path.exists(save_path):
            continue

        im2 = im.crop(clip[i])
        im2.save(save_path, "PNG")"""

    save_path = os.path.join(SAVE_1080_FOLDER, f)
    if skip_existing and os.path.exists(save_path):
        return

    im = Image.open(os.path.join(SAVE_4K_FOLDER, f))
    target_size = (1920, 1080)
    im = im.resize(target_size)
    im.save(save_path, "PNG")


with Pool(4) as p:
    p.map(f_1080, os.listdir(SAVE_4K_FOLDER))

In [9]:
def fix_1080(f):
    if f.startswith('.'):
        return
    try:
        save_path = os.path.join(SAVE_4K_FOLDER, f)
        im = Image.open(save_path)
    except Exception as e:
        print(e)

with Pool(6) as p:
    p.map(fix_1080, os.listdir(SAVE_4K_FOLDER))

In [7]:
def f_270(f):
    if f.startswith('.'):
        return

    save_path = os.path.join(SAVE_270_FOLDER, f)
    if skip_existing and os.path.exists(save_path):
        return

    im = Image.open(os.path.join(SAVE_4K_FOLDER, f))
    target_size = (480, 270)
    im = im.resize(target_size)
    im.save(save_path, "PNG")


with Pool(6) as p:
    p.map(f_270, os.listdir(SAVE_4K_FOLDER))